## Функции
**return** использовать не обязательно, функция по умолчанию вернет **None**

In [1]:
def foo():
    x = 42
    
print(foo())    

None


Функции можно документировать: 

In [4]:
def foo():
    """Some doc"""
    
foo.__doc__    
foo?

In [6]:
def min(x, y):
    return x if x < y else y

# min(x = -5, z = 3)
min(1, 2)

1

Функция может принимать произвольное число аргументов:

In [12]:
def min(*args): #type(args) == tuple
    res = float('inf') #!!!!!!
    for arg in args:
        if arg < res:
            res = arg
    return res

print(min(1,2,-1))
print(min())

-1
inf


Можно применять функции к коллекции:

In [15]:
xs = {-5, 1, 2}
print(min(*xs))
print(min(*[1, 2, 3]))
print(min(*(1,2,3)))

-5
1
1


Значения по умолчанию: вычисляются и ассоциируются только один раз – в момент объявления функции.
* рекомендация -- не использовать изменяемые типы для значений по умолчанию

In [16]:
def bounded_min(first, *args, lo = float('-inf'), hi = float('inf')):
    res = hi
    for arg in (first, ) + args:
        if arg < res and lo < arg < hi:
            res = arg
    return max(res, lo)

bounded_min(-5, 12, 13, lo=0, hi=255)

12

In [23]:
def unique(iterable, seen = set()):
    acc = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            acc.append(item)
    return acc

xs = [1, 1, 2, 3]
print(unique(xs))
print(unique(xs))
print(unique.__defaults__)

# вариант исправления
def unique(iterable, seen = None):
    seen = set(seen or []) #None - falsy value
    acc = []
    for item in iterable:
        if item not in seen:
            seen.add(item)
            acc.append(item)
    return acc

xs = [1, 1, 2, 3]
print(unique(xs))
print(unique(xs))
print(unique.__defaults__)

print({1, 2} or [])
print(None or [])

[1, 2, 3]
[]
({1, 2, 3},)
[1, 2, 3]
[1, 2, 3]
(None,)
{1, 2}
[]


Можно потребовать, чтобы часть аргументов **всегда** передавалась как ключевые:

In [29]:
def flatten(xs, *, depth = None):
    pass

print(flatten([1,2,[3],[3,[4],5]], depth = 1))
print(flatten([1,2,[3]], 2)) #flatten() takes 1 positional argument but 2 were given

None


TypeError: flatten() takes 1 positional argument but 2 were given

In [31]:
def runner(cmd, **kwargs):
    pass

runner('abc', limit=42)
runner('abc', **{'limit': False})
options = {'abc' : 1}
runner('abc', **options)

## Упаковка и распаковка

In [36]:
acc = []
seen = set()
(acc, seen) = ([], set())

x, y, z = [1, 2, 3]
print('%s %s %s' % (x, y, z))
x, y, z = {3, 1, 2}
print('%s %s %s' % (x, y, z))
x, y, z = 'xyz'
print('%s %s %s' % (x, y, z))


#можно и со скобочками
rectangle = (0, 0), (4, 4)
(x1, y1), (x2, y2) = rectangle

1 2 3
1 2 3
x y z


In [37]:
#можно распоковать только часть
first, *rest = range(1, 5)
first, rest

(1, [2, 3, 4])

In [39]:
first, *rest, last = range(1, 5)
first, rest, last

(1, [2, 3], 4)

In [40]:
first, *rest, last = [42]

ValueError: not enough values to unpack (expected at least 2, got 1)

In [44]:
*_, (first, *rest) = [range(1, 5)] * 5
print([range(1, 5)] * 5)
print(first, rest)

[range(1, 5), range(1, 5), range(1, 5), range(1, 5), range(1, 5)]
1 [2, 3, 4]


In [46]:
print([range(4), range(2)])
for a, *b in [range(4), range(2)]:
    print(b)

[range(0, 4), range(0, 2)]
[1, 2, 3]
[1]


### Распаковка в байткод

In [47]:
import dis
dis.dis("first, *rest, last = ('a', 'b', 'c')")

  1           0 LOAD_CONST               4 (('a', 'b', 'c'))
              3 UNPACK_EX              257
              6 STORE_NAME               0 (first)
              9 STORE_NAME               1 (rest)
             12 STORE_NAME               2 (last)
             15 LOAD_CONST               3 (None)
             18 RETURN_VALUE


In [48]:
x, (x, y) = 1, (2, 3)
x

2

Синтаксически схожие конструкции могут иметь различную семантику времени исполнения

In [49]:
dis.dis("first, *rest, last = ['a', 'b', 'c']")

  1           0 LOAD_CONST               0 ('a')
              3 LOAD_CONST               1 ('b')
              6 LOAD_CONST               2 ('c')
              9 BUILD_LIST               3
             12 UNPACK_EX              257
             15 STORE_NAME               0 (first)
             18 STORE_NAME               1 (rest)
             21 STORE_NAME               2 (last)
             24 LOAD_CONST               3 (None)
             27 RETURN_VALUE


## Scopes (области видимости)
Функции в Python -- это объекты первого класса (first class citizens), т.е. с ними можно делать все то, что и с другими значениями

In [51]:
def wrapper():
    def identity(x):
        return x
    return identity

f = wrapper()
f(42)

42

In [52]:
def make_min(*, lo, hi):
    def inner(first, *args):
        res = hi
        for arg in (first, ) + args:
            if arg < res and lo < arg < hi:
                res = arg
        return max(res, lo)
    return inner

bounded_min = make_min(lo = 0, hi = 255)
bounded_min(-5, 12 ,13)

12

Всего существует 4 области видимости (**LEGB**): поиск от локальной к встроеннной
   * local
   * enclosing
   * global
   * built-in

In [57]:
min #built-in
min = 42 # global
#print(globals())
def f(*args):
    min = 2
    def g(): #enclosing
        min = 4 #local
        print(min)
        print(locals())
        
f(1)        

Можно использовать переменные, определенные во внешних областях видимости.

Это работает, так как поиск переменных осуществляется во время исполнения функции, а не во время объявления.

In [58]:
def f():
    print(i)
    
for i in range(4):
    f()

0
1
2
3


Для присваивания правило LEGB не работает.

Для изменения этого поведения можно использовать операторы **global** и **nonlocal**

In [67]:
min = 42
def f():
    min += 1 #имя будет создано в локальной области видимости
    return min

f()

UnboundLocalError: local variable 'min' referenced before assignment

In [65]:
min = 42
def f():
    global min 
    min += 1 #будет использована глобальная переменная
    return min

f()

43

In [68]:
def cell(value = None):
    def get():
        return value
    def set(update):
        nonlocal value
        value = update
    return get, set

get, set = cell()
set(42)
get()

42

# Правила поиска переменных
``
LEGB(Local -> Enclosed -> Global -> Built-in):
    Local can be inside a function or class method, for example.
    Enclosed can be its enclosing function, e.g., if a function is wrapped inside another function.
    Global refers to the uppermost level of the executing script itself, and
    Built-in are special names that Python reserves for itself.
``

## Элементы функционального программирования
* Python -- не функциональный язык, но в нем есть элементы функционального программирования
* Можно объявлять анонимные функции (lambda)

In [77]:
square = lambda x : x * 2
map(square, range(4))
print(list(map(square, range(4))))

[0, 2, 4, 6]


In [80]:
filter(lambda x: x % 2 != 0, range(10))
list(filter(lambda x: x % 2 != 0, range(10)))

[1, 3, 5, 7, 9]

Вместо предиката можно передать None, в этом случае в последовательности останутся только truthy значения

In [82]:
xs = [0, None, [], {}, "", 42]
list(filter(None, xs))

[42]

### zip 
* строит последовательность кортежей элементов нескольких последовательностей
* поведение в случае последовательностей различной длины аналогично **map**

In [84]:
list(zip('abc', range(10)))

[('a', 0), ('b', 1), ('c', 2)]

In [85]:
list(zip('abc', range(3), [42j, 42j, 42j]))

[('a', 0, 42j), ('b', 1, 42j), ('c', 2, 42j)]

In [87]:
[x ** 2 for x in range(10) if x % 2 == 1]

[1, 9, 25, 49, 81]

In [88]:
(x ** 2 for x in range(10) if x % 2 == 1)

<generator object <genexpr> at 0x105c3a570>

### сравнения
* объекты == и !=
* singleton (**None**) - **is** или **is not**
* для bool используйте сам объект или оператор **not**
* отсутствие элемента в словаре с помощью оператора **not in**